In [2]:
import pandas as pd
from pytorch_pretrained_bert import BertTokenizer, BertConfig,BertForSequenceClassification,BertAdam
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset,RandomSampler,DataLoader,SequentialSampler
from tqdm import trange
import numpy as np


In [4]:
df = pd.read_csv('./entries.train.csv', header= 0,
                        encoding= 'unicode_escape')
df=df.drop(df.iloc[:,[1,2,3]],axis=1)
df = df[df['Good luck on your new start !'].notna()]
df.columns = ['Number_of_corrections', 'Original_sentence', 'Correction_1', 'Correction_2',
              'Correction_3', 'Correction_4', 'Correction_5', 'Correction_6', 'Correction_7', 'Correction_8']
df = df.fillna('')

C:\Users\nel_1\AppData\Local\Temp\ipykernel_30064\3432210751.py:1: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./entries.train.csv', header= 0,


In [21]:
training_data = []
has_correction_column = []  #labels

for _, row in df.iterrows():
    original_sentence = row['Original_sentence']
    num_corrections = int(row['Number_of_corrections'])
    
    # If no corrections, include the original sentence 
    if num_corrections == 0:
        training_data.append((original_sentence, original_sentence))
        has_correction_column.append(0)  # No corrections
    else:
        for i in range(1, num_corrections + 1):
            corrected_sentence = row[f'Correction_{i}']
            if corrected_sentence: 
                training_data.append((original_sentence, corrected_sentence))
                has_correction_column.append(1)  # There is correction


training_df = pd.DataFrame(training_data, columns=["Input", "Output"])

training_df['Has_Correction'] = has_correction_column
training_true = training_df.to_csv('./training_true.csv',index=False)

In [22]:
df_end = pd.read_csv('./training_true.csv', header= 0,
                        encoding= 'unicode_escape')
df_end.rename(columns={"Has_Correction":"label"},inplace=True)
train_df = df_end.sample(frac=0.5, random_state=42)

In [24]:
sentences = df_end.Input.values
sentences = ["[CLS]" + sentence + " [SEP]" for sentence in sentences]
labels = df_end.label.values

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("Tokenize version of the the first sentence:")
print(tokenized_texts[0])

Tokenize version of the the first sentence:
['[', 'cl', '##s', ']', 'ur', '##l', '[SEP]']


In [26]:
MAX_LEN = 128

# Pading
input_ids = pad_sequences(
    [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], 
    maxlen=MAX_LEN, dtype="long", truncating="post", padding="post"
    )
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype ="long", truncating="post",padding ="post")


In [27]:
# attention masks
attention_masks = []

for seq in input_ids:
  seq_mask = [float(i > 0) for i in seq]
  attention_masks.append(seq_mask)


In [28]:


train_inputs, validation_inputs, train_labels, validation_labels = \
  train_test_split(input_ids, labels, random_state=2018, test_size=0.1)

train_masks, validation_masks , _, _ = \
  train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)
  

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [29]:
#Training
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = \
  DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = \
  TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = \
  DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [30]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                      num_labels=2)

model.cuda()

c:\Users\nlp-user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_pretrained_bert\modeling.py:603: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(next(model.parameters()).device)

cuda:0


In [32]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']

optimizer_grouped_parameters = [
    {
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01
    },
    {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
        'weight_decay_rate': 0.00
    }
  ]

optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-5, warmup=0.1)

t_total value of -1 results in schedule not being applied


In [33]:
def flat_accuracy(preds, labels):
  pred_flat  = np.argmax(preds , axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat)/len(labels_flat)


In [34]:
train_loss_set = []
epochs = 4

for _ in trange(epochs, desc="Epoch"):

  model.train()
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  for step, batch in enumerate(train_dataloader):
    
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask , b_labels = batch
    optimizer.zero_grad()
    loss = model(b_input_ids, token_type_ids=None, 
                 attention_mask=b_input_mask, labels=b_labels)
    
    train_loss_set.append(loss.item())
    loss.backward()
    optimizer.step()
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0


for batch in validation_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  

  with torch.no_grad():
    logits = model(b_input_ids, token_type_ids =None, attention_mask=b_input_mask)
    
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:  25%|██▌       | 1/4 [1:43:52<5:11:36, 6232.01s/it]

Train loss: 0.5451054029886059


Epoch:  50%|█████     | 2/4 [3:27:47<3:27:48, 6234.29s/it]

Train loss: 0.47969428329975744


Epoch:  75%|███████▌  | 3/4 [5:11:42<1:43:54, 6234.30s/it]

Train loss: 0.4003050917683691


Epoch: 100%|██████████| 4/4 [6:55:40<00:00, 6235.03s/it]  

Train loss: 0.3145309952549118


Validation Accuracy: 0.7285538307646711


In [36]:

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in validation_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  
  with torch.no_grad():

    logits = model(b_input_ids, token_type_ids =None, attention_mask=b_input_mask)
    
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


Validation Accuracy: 0.7285538307646711
